# Sounding on DBpedia and BnF.Data

In this notebook, we will show a query about the number of lawyers and economists on DBpedia and BnF.Data.

Unfortunately, we can't run it because the SERVICE clause doesn't works on DBPedia and BnF.Data endpoints. So we will use GraphDB who permits to use them with the SERVICE clause.

## Economist

First, we study the number of economists by performing a query that aggregates data from BnF.Data and DBpedia. On DBpedia, we combine the data available from the Wikipedia site and the use of their ontology. Furthermore, the data between DBpedia and BnF.data are aligned.

In [ ]:
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>

PREFIX  egr:  <http://rdvocab.info/ElementsGr2/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?s ?name ?prefLabel ?year
WHERE
  {
     {SERVICE <https://data.bnf.fr/sparql>
          {SELECT DISTINCT ?s ?name ?prefLabel ?year
           WHERE
    
                  {?s egr:biographicalInformation ?bio .
                   ?s egr:dateOfBirth ?bd .
                 
                   BIND(STRBEFORE(STRAFTER(STR(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
                   FILTER ( ?year > "1800" )
                   FILTER ( regex(?bio, "économiste", "i") || regex(?bio, "Economist", "i") || regex(?bio, "professeur d'économie", "i") || regex(?bio, "économie", "i") ||regex(?bio, "docteur en économie", "i"))
                
                   OPTIONAL {?s foaf:name ?name}
                   OPTIONAL {?s skos:prefLabel ?prefLabel}
                  }
          }
    }
    UNION
    {SERVICE <https://DBpedia.org/sparql>
          {SELECT DISTINCT ?s ?name (year(xsd:dateTime(?Birth_Date)) AS ?year)
           WHERE
                { 
                      {?s a dbo:Economist .
                       ?s dbp:birthDate ?Birth_Date .
                      FILTER (xsd:date (?Birth_Date) > "1800-01-01"^^xsd:date)
                      OPTIONAL{?s foaf:name ?name .}

                      }
                     UNION
                     {?s ?propriety dbr:Economist .
                      ?s dbp:birthDate ?Birth_Date .
                     FILTER (xsd:date (?Birth_Date) > "1800-01-01"^^xsd:date)
                     OPTIONAL{?s foaf:name ?name .}
                     }
               }
          }
    }
  }
ORDER BY ?year
"""

In running the query, we find  4,624 results who could that could fit into the definition of economist, although some results are duplicated because some people have two labels and one is empty.

## Lawyer

In [ ]:
Secondly, we study the number of lawyer. 

In [4]:
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX rdagroup2elements: <http://rdvocab.info/uri/schema/FRBRentitiesRDA>
PREFIX  egr:  <http://rdvocab.info/ElementsGr2/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?s ?name ?prefLabel ?year
WHERE
{
    {SERVICE <https://data.bnf.fr/sparql>
         {SELECT DISTINCT ?s ?name ?prefLabel ?year
           WHERE
    
                  {?s egr:biographicalInformation ?bio .
                   ?s egr:dateOfBirth ?bd
                   BIND(STRBEFORE(STRAFTER(STR(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
                   FILTER(?year > "1800")
                   FILTER ( regex(?bio, "juriste", "i") || regex(?bio, "Jurist", "i") || regex(?bio, "professeur de droit", "i") || regex(?bio, "Lawyer", "i") ||regex(?bio, "docteur en droit", "i"))
                   OPTIONAL {?s foaf:name ?name}
                   OPTIONAL {?s skos:prefLabel ?prefLabel}
                  }
          }
    }
   UNION
   {
    SERVICE <https://DBpedia.org/sparql>
       {SELECT DISTINCT ?s ?name (year(xsd:dateTime(?Birth_Date)) AS ?year)
          WHERE
             {
                 {?s ?propriety dbr:Jurist .
                  ?s dbp:birthDate ?Birth_Date .
                  FILTER (xsd:date (?Birth_Date) > "1800-01-01"^^xsd:date)
                  OPTIONAL{?s foaf:name ?name .
                  FILTER (xsd:string(?name) !="Marcus Tullius Cicero")
                          }
                 }
                UNION
                 {?s a dbo:Lawyer .
                  ?s dbp:birthDate ?Birth_Date .
                  FILTER (xsd:date (?Birth_Date) > "1800-01-01"^^xsd:date)
                  OPTIONAL{?s foaf:name ?name .
                  FILTER (xsd:string(?name) !="Marcus Tullius Cicero")
                           }    
                 }
                UNION
                 {?s a dbo:Professor .
                  ?s dbp:birthDate ?Birth_Date .
                  ?s owl:sameAs ?sameAs .
                  ?s dbo:abstract ?abstract .
                  FILTER (CONTAINS(?abstract, "Jurist") || CONTAINS(?abstract, "Lawyer") || CONTAINS(?abstract, "lawyer")  || CONTAINS(?abstract, "jurist")|| CONTAINS(?abstract, "juriste")|| CONTAINS(?abstract, "Juriste"))
                  OPTIONAL{?s foaf:name ?name .
                  FILTER (xsd:string(?name) !="Marcus Tullius Cicero")
                           }
                  FILTER (xsd:date (?Birth_Date) > "1800-01-01"^^xsd:date)
                  }
             }
        }
   }
}

ORDER BY ?year
"""

Running the query, we find 6,209 results that could fit into the Jurist/Lawyer definition, although again some results are duplicated.

Then, we tried to combine both queries but GraphDB would not display them, certainly because the query is too heavy.